In [1]:
import open3d as o3
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader,random_split
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the first GPU
    print("GPU available.")
else:
    device = torch.device("cpu")
    print("Using CPU.")

GPU available.


In [3]:
def visualize_cloud(clean_lidar_points):
    # Convert the numpy array to a Vector3dVector
    point_cloud = o3.utility.Vector3dVector(clean_lidar_points)
    # You can now use point_cloud with other Open3D functions and methods
    pcd = o3.geometry.PointCloud()
    pcd.points = point_cloud
    o3.visualization.draw_geometries([pcd])

In [4]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()

        # Encoder
        self.encoder =nn.Sequential(
        nn.Conv1d(3, 16, kernel_size=3, stride=2, padding=1),
        nn.ReLU(True),
        nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1),
        nn.ReLU(True),
        nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1),
        nn.ReLU(True)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose1d(16, 3, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [5]:
model = DenoisingAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.load_state_dict(torch.load('denoise_model.pth'))
model.eval()

DenoisingAutoencoder(
  (encoder): Sequential(
    (0): Conv1d(3, 16, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): ReLU(inplace=True)
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(2,), padding=(1,))
    (3): ReLU(inplace=True)
    (4): Conv1d(32, 64, kernel_size=(3,), stride=(2,), padding=(1,))
    (5): ReLU(inplace=True)
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
    (1): ReLU(inplace=True)
    (2): ConvTranspose1d(32, 16, kernel_size=(4,), stride=(2,), padding=(1,))
    (3): ReLU(inplace=True)
    (4): ConvTranspose1d(16, 3, kernel_size=(4,), stride=(2,), padding=(1,))
    (5): ReLU(inplace=True)
  )
)

In [57]:
clean_lidar_data = np.load("sample_input.npz")['lidar']

In [58]:
visualize_cloud(clean_lidar_data)

In [60]:
input = torch.tensor(clean_lidar_data, dtype=torch.float).to(device).unsqueeze(0).transpose(1, 2)
output = model(input)
output=output.transpose(2, 1)[0, :-1, :].cpu().detach().numpy()
visualize_cloud(output)